# Loaders

 

Just to import the differents modules that will interest us

In [ ]:
import pandas as pd
import json

In [ ]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=True)
import plotly.graph_objs as go
import plotly.offline as py
import plotly.graph_objs as go

# Data Loaders

Here I decided to concat all the days together, as the data contains the timestamp.

This is done with :

 

cat data/2018-* > all.txt

In [ ]:
with open('./hndata/all.txt', encoding='utf-8') as fp:
    searches= fp.readlines()


In [ ]:
import re
import json
garbage=[]
def load_dirty_json(dirty_json):
    regex_replace = [(r"([ \{,:\[])(u)?'([^']+)'", r'\1"\3"'), (r" False([, \}\]])", r' false\1'), (r" True([, \}\]])", r' true\1')]
    for r, s in regex_replace:
        dirty_json = re.sub(r, s, dirty_json)
    try:
        clean_json = json.loads(dirty_json)
    except json.decoder.JSONDecodeError:
        garbage.append(dirty_json)
        return dict()

    return clean_json

searches_dict=[load_dirty_json(search) for search in searches]
df=pd.DataFrame(searches_dict)
df.dropna(how='all',inplace=True) # Where  all the columns are null, we drop the line
del searches_dict
df.head()


I realised that some of the data in the files are ill-formed, throwing json error. I made the garbage list to explore why

In [ ]:

len(df)


So, we have roughly 880k request spread around a full month. 

 

# Timeseries

Let's plot the associated time-series, just in order to have a feel of it

First, we will set the timestamp as a Datetime, then set it as an index

In [ ]:

df['timestamp']=pd.to_datetime(df['timestamp'])
ts_df=df.set_index(['timestamp']).sort_index()
ts=ts_df.index.value_counts().sort_index()
ts=ts.resample('1H').sum().fillna(0)
# Here depends if you have plotly extenstion loaded. The commented line will make the plot in a new tab, otherwise in the notebook.
#py.plot(ts.iplot( fill=True, filename='Timeseries of the search count per hour',asFigure=True))
ts.iplot( fill=True, filename='Timeseries of the search count per hour')


In [ ]:

ts.describe()


This looks like a standard time series, where we can clearly see the weekdays and the week-end.

The mean request per hour is around 1182, with a standard deviation of 362.

However, we can see as well some sort of anomalies, during the 14th of June, where we can reach up to 4811 request per hour. We will explore this later

 

# Exploration

In [ ]:

df.app_id.unique()


So, we have only one app, wich makes sense. Therefore, we can drop this column.

In [ ]:

df.drop(['app_id'], axis=1,inplace=True)


## Query related exploration

 

First, let's explore the user-queries.

We can derive two simple metrics : the length of the query word-wise and character-wise

In [ ]:
df['qlen']=df['query'].apply(lambda x: len(x.split(' '))) # This is a really "naive" tokenizer, but as the query are usually short, this can do the trick
df['qcharlen']=df['query'].apply(lambda x: len(x))

In [ ]:

df.head()


#### Word-wise length Histogram

In [ ]:

#py.plot(df.qlen.iplot(kind='histogram',asFigure=True))
df.qlen.iplot(kind='histogram')

 


Here as well, we can see a long tail, hence the histogram being a bit squashed on the left. However, this is not the expected behavior for queries, as it should be less than 5 words on average. Let's check those "anomalous" queries

In [ ]:

df[(df.qlen>100)].head()


We can see that the number of words is almost the same as the number of char.

This can mean only one thing

In [ ]:

df[(df.qlen>100)]['query'].unique()


So, for all the words with more than 100 words, most of them are actually whitespace. This is the limitation of the tokenizer by space used before, however we can agree that these queries are not really relevant. Let's remove them as well

In [ ]:
df=df[(df.qlen<40)]
#py.plot(df.qlen.iplot(kind='histogram',asFigure=True))
df.qlen.iplot(kind='histogram')

 


#### Character-wise length Histogram

In [ ]:

#py.plot(df.qcharlen.iplot(kind='histogram',asFigure=True))

df.qcharlen.iplot(kind='histogram')



In [ ]:
df[df.qcharlen==0].head()

Two thoughts here : First, there is a lot of empty queries with no length.

The strange part is, if you look at the datagrame above, that even though the queries are empty, on the same day, we have various hits numbers. This shows maybe some inconsistency in the input data.

As they are pointless if we want to study query statistics, we remove them from the main dataframe.

Furthermore, let's analyze the query with only 1 characters

In [ ]:
zeros_char_df=df[df.qcharlen==0]
df = df[~(df.qcharlen==0)]
df[df.qcharlen==1]['query'].unique()

Once again, these queries should be more considered as noise, the unique letters will probably not provide any satisficient results, nor the ideograms.

In [ ]:
df = df[~(df.qcharlen==1)]
#py.plot(df.qcharlen.iplot(kind='histogram',asFigure=True))
df.qcharlen.iplot(kind='histogram')

We can see a heavy tail distribution here, most of the queries being lower than 80 characters long, with a huge gap at 80.

Let's explore these

In [ ]:
df[df.qcharlen>80].head()

What we can see here, is "regular" queries (that looks like sentences), but as well urls and cached urls.

In [ ]:
urls_df=df[((df['query'].str.startswith('http:')) | (df['query'].str.startswith('https:'))| (df['query'].str.startswith('cache:')))]
df=df[~((df['query'].str.startswith('http:')) | (df['query'].str.startswith('https:'))| (df['query'].str.startswith('cache:')))]
print('Number of queries that are urls: '+str(len(urls_df)))
urls_df.head()

## Click analysis

In [ ]:
df['numcliks']=df.clicks.fillna('').apply(lambda x: len(x))
#py.plot(df.numcliks.iplot(kind='histogram',asFigure=True))
df.numcliks.iplot(kind='histogram')

What we can see from the clicks pattern is that most of the searches do not lead to any clicks. If we take a sample of the queries that are not clicked, these look like a normal queries

In [ ]:
df[df.numcliks==0]['query'].head(10)

In [ ]:
df[df.numcliks==0].nb_hits.describe()

When looking at the description, we see that the first quartile of hits are less or equals than 2. Therefore, the lack of clicks cannot be imputed to the lack of results.

In [ ]:
#py.plot(df[df.numcliks==0].nb_hits.iplot(kind='histogram',asFigure=True))
df[df.numcliks==0].nb_hits.iplot(kind='histogram')

# Top queries

 

## Top queries per day

 

Let's see what are the trends in queries, the top queries and so on

In [ ]:
df.groupby(['query'])['query'].count().sort_values(ascending=False)[0:20]

And let's see if we groupby the queries by number of hits

In [ ]:
df.groupby(['nb_hits'])['query'].count().sort_values(ascending=False)[0:20]

Something is odd here, as 111426 seems to be really specific number of hit, and is clearly an outlier here. One could probably calculate the probability of having differents queries yielding this number of hits, but my guess is that this is the same query happening a lot. Let's see

In [ ]:
df[df.nb_hits==111426]['query'].unique()

As expected, it's a unique query that generated this specific number. Let's plot the timeseries of this query

In [ ]:
tes=df[df['query']=='tes']
tes['timestamp']=pd.to_datetime(tes['timestamp'])
ts=tes.set_index(['timestamp']).sort_index()
ts=ts.index.value_counts().sort_index()
ts=ts.resample('1H').sum().fillna(0)
#py.plot(ts.iplot( fill=True, filename='Timeseries of the search -tes- per hour',asFigure=True))
ts.iplot(fill=True, filename='Timeseries of the search -tes- per hour')


This is the interesting part. The query 'tes' was actually trending on the 14th of June, resulting in a lot of queries that day but almost none the other days.

Maybe it's possible to build a feature, where we detect the trends of days, meaning not the top query of the day, but what was trending that day, but was not the other days.

This will be done in the notebook Trends.ipynb